In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
data = pd.read_csv("Aus_Brisbane_Suburbs_LatLong_Wiki.csv") 
data.head()

,Suburb Type,Suburb,Lat,Long
0,Inner suburbs,Bardon,NaN,NaN
1,Inner suburbs,Bowen Hills,NaN,NaN
2,Inner suburbs,Brisbane,NaN,NaN
3,Inner suburbs,East Brisbane,NaN,NaN
4,Inner suburbs,Fortitude Valley,NaN,NaN


In [3]:
for add in data['Suburb']:
    address=add
    geolocator = Nominatim(user_agent="aus_explorer")
    location = geolocator.geocode(address+', Brisbane')
    if location:
        latitude = location.latitude
        longitude = location.longitude
        data.loc[data.Suburb==address,['Lat','Long']]=latitude,longitude
data.head()

,Suburb Type,Suburb,Lat,Long
0,Inner suburbs,Bardon,-27.459397,152.980339
1,Inner suburbs,Bowen Hills,-27.446675,153.040222
2,Inner suburbs,Brisbane,-27.464860,153.009605
3,Inner suburbs,East Brisbane,-27.483250,153.045574
4,Inner suburbs,Fortitude Valley,-27.456247,153.035818


In [4]:
data.dropna(inplace=True)

In [5]:
data.to_csv('Aus_Brisbane_Suburbs_LatLong_Final.csv', encoding='utf-8', index=False)


In [6]:
address = 'Brisbane, Australia'

geolocator = Nominatim(user_agent="aus_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brisbane  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brisbane  are -27.4689682, 153.0234991.


In [7]:
# create map of Brisbane using latitude and longitude values
map_brisbane = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, suburb, neigh in zip(data['Lat'], data['Long'], data['Suburb'],data['Suburb Type']):
    label = '{},{}'.format(neigh,suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brisbane)  
    
map_brisbane

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'ZIINEFEUWNYFAXEWX2GQ4QQ0NSWLPVPAJFGI1IOOPOV3AD5R' # your Foursquare ID
CLIENT_SECRET = 'EVCMJ02U4ZDSLE5GPLYSSUGNHO0OVEZ3BY5IEIN1VIA4HGGF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZIINEFEUWNYFAXEWX2GQ4QQ0NSWLPVPAJFGI1IOOPOV3AD5R
CLIENT_SECRET:EVCMJ02U4ZDSLE5GPLYSSUGNHO0OVEZ3BY5IEIN1VIA4HGGF


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
venues = getNearbyVenues(names=data['Suburb'],
                                   latitudes=data['Lat'],
                                   longitudes=data['Long']
                                  )


Bardon
Bowen Hills
Brisbane
East Brisbane
Fortitude Valley
Herston
Highgate Hill
Kangaroo Point
Kelvin Grove
New Farm
Newstead
Paddington
Petrie Terrace
Red Hill
South Brisbane
Spring Hill
Teneriffe
West End
Woolloongabba
Albion
Alderley
Ascot
Aspley
Bald Hills
Banyo
Boondall
Bracken Ridge
Bridgeman Downs
Brighton
Brisbane Airport
Carseldine
Chermside
Chermside West
Clayfield
Deagon
Eagle Farm
Everton Park
Fitzgibbon
Gaythorne
Geebung
Gordon Park
Grange
Hamilton
Hendra
Kedron
Keperra
Lutwyche
McDowall
Mitchelton
Myrtletown
Newmarket
Northgate
Nudgee
Nudgee Beach
Nundah
Pinkenba
Sandgate
Shorncliffe
Stafford
Stafford Heights
Taigum
Virginia
Wavell Heights
Wilston
Windsor
Wooloowin
Zillmere
Acacia Ridge
Algester
Annerley
Archerfield
Burbank
Calamvale
Coopers Plains
Darra
Doolandella
Drewvale
Durack
Dutton Park
Eight Mile Plains
Ellen Grove
Fairfield
Forest Lake
Greenslopes
Heathwood
Holland Park
Holland Park West
Inala
Karawatha
Kuraby
Larapinta
MacGregor
Mackenzie
Mansfield
Moorooka
Mou

In [16]:
venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bardon,-27.459397,152.980339,Goodlife Health Club,-27.455670,152.980657,Gym
1,Bardon,-27.459397,152.980339,Beans On The Green,-27.456356,152.978202,Café
2,Bardon,-27.459397,152.980339,Bardon bowls club,-27.456413,152.978282,Bowling Green
3,Bardon,-27.459397,152.980339,Bardon Foodstore,-27.463457,152.979592,Convenience Store
4,Bowen Hills,-27.446675,153.040222,Rogue Bar + Bistro,-27.445979,153.043879,Bistro
5,Bowen Hills,-27.446675,153.040222,Di Bella Coffee HQ,-27.443332,153.038579,Coffee Shop
6,Bowen Hills,-27.446675,153.040222,Betty’s Burgers & Concrete Co.,-27.449814,153.043793,Burger Joint
7,Bowen Hills,-27.446675,153.040222,The Boo,-27.444699,153.042515,Café
8,Bowen Hills,-27.446675,153.040222,Homemaker The Valley North,-27.450098,153.040381,Shopping Mall
9,Bowen Hills,-27.446675,153.040222,Merlo,-27.442231,153.040401,Coffee Shop


In [17]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acacia Ridge,4,4,4,4,4,4
Albion,1,1,1,1,1,1
Alderley,6,6,6,6,6,6
Annerley,16,16,16,16,16,16
Anstead,1,1,1,1,1,1
Archerfield,1,1,1,1,1,1
Ascot,16,16,16,16,16,16
Ashgrove,1,1,1,1,1,1
Aspley,22,22,22,22,22,22


In [18]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,Accessories Store,Advertising Agency,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Child Care Service,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,RV Park,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Souvlaki Shop,Spa,Sporting Goods Shop,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bardon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bardon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Accessories Store,Advertising Agency,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Child Care Service,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,RV Park,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Souvlaki Shop,Spa,Sporting Goods Shop,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Acacia Ridge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.25

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acacia Ridge,Pool,Pub,Sandwich Place,Fast Food Restaurant,Women's Store,Event Space,Football Stadium,Food Truck,Food Court,Flea Market
1,Albion,Playground,Fruit & Vegetable Store,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court,Flea Market,Fish Market,Fish & Chips Shop
2,Alderley,Supermarket,Thrift / Vintage Store,Electronics Store,Grocery Store,Train Station,Thai Restaurant,Women's Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
3,Annerley,Café,IT Services,Supermarket,Grocery Store,Dance Studio,Pharmacy,Comic Shop,Coffee Shop,Pub,Camera Store
4,Anstead,Café,Fruit & Vegetable Store,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court,Flea Market,Fish Market,Fish & Chips Shop


In [22]:
# set number of clusters
kclusters = 10

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([8, 0, 7, 8, 6, 9, 6, 8, 8, 8], dtype=int32)

In [26]:

merged = data

merged.rename(columns={'Suburb': 'Neighborhood'}, inplace=True)
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Suburb Type,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Inner suburbs,Bardon,-27.459397,152.980339,6.0,Café,Convenience Store,Bowling Green,Gym,Women's Store,French Restaurant,Football Stadium,Food Truck,Food Court,Flea Market
1,Inner suburbs,Bowen Hills,-27.446675,153.040222,8.0,Coffee Shop,Gym,Shopping Mall,Clothing Store,Sandwich Place,Brewery,Bistro,Burger Joint,Supermarket,Café
2,Inner suburbs,Brisbane,-27.464860,153.009605,8.0,Hostel,Brewery,Indian Restaurant,Bar,Tapas Restaurant,Multiplex,Café,Mexican Restaurant,Burger Joint,Football Stadium
3,Inner suburbs,East Brisbane,-27.483250,153.045574,8.0,Shop & Service,American Restaurant,Sandwich Place,Brewery,Sushi Restaurant,Women's Store,Farmers Market,French Restaurant,Football Stadium,Food Truck
4,Inner suburbs,Fortitude Valley,-27.456247,153.035818,8.0,Coffee Shop,Bar,Japanese Restaurant,Cocktail Bar,Vietnamese Restaurant,Burger Joint,Hotel,Italian Restaurant,Pub,Café


In [27]:
merged.dropna(inplace=True)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Lat'], merged['Long'], merged['Neighborhood'], merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
merged[merged['Cluster Labels']==2.0]

,Suburb Type,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Inner suburbs,Newstead,-27.442823,153.042686,2.0,Coffee Shop,Beer Garden,Asian Restaurant,Café,Steakhouse,Music Venue,Park,Bistro,Gym,Food Truck
16,Inner suburbs,Teneriffe,-27.455924,153.046443,2.0,Coffee Shop,Café,Thai Restaurant,Fish Market,Park,Ice Cream Shop,Hotel,Breakfast Spot,Brewery,Italian Restaurant
51,Northern suburbs,Northgate,-27.390816,153.069353,2.0,Coffee Shop,Bakery,Women's Store,Farmers Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
72,Southern suburbs,Calamvale,-27.616947,153.049530,2.0,Ice Cream Shop,Baby Store,Women's Store,Farmers Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
79,Southern suburbs,Eight Mile Plains,-27.577918,153.092299,2.0,Office,Coffee Shop,Electronics Store,Café,Women's Store,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
88,Southern suburbs,Karawatha,-27.618629,153.092046,2.0,Coffee Shop,Women's Store,Fabric Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court,Flea Market,Fish Market
126,Eastern suburbs,Carina Heights,-27.501451,153.096698,2.0,Department Store,Coffee Shop,Italian Restaurant,Thai Restaurant,Women's Store,Farmers Market,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
182,Western suburbs,Sherwood,-27.530861,152.982610,2.0,Café,Coffee Shop,Bakery,Grocery Store,Video Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Women's Store
